# Imports and Config

In [1]:
import pandas as pd

# Load Data

In [ ]:
raw = pd.read_csv("../data/biored_raw.csv")

...

In [1]:
import stanza


stanza.download('de')
nlp = stanza.Pipeline('de', processors='tokenize,pos,lemma,depparse')


text = "Angela Merkel war Präsidentin."

doc = nlp(text)

rows = []
for sent_id, sentence in enumerate(doc.sentences, start=1):
    for w in sentence.words:
        rows.append({
            "sentence_id": sent_id,
            "token_id": w.id,
            "text": w.text,
            "lemma": w.lemma,
            "upos": w.upos,
            "xpos": w.xpos,
            "feats": w.feats,
            "head": w.head,
            "deprel": w.deprel,
        })

df = pd.DataFrame(rows)
df

/Users/paulsilberbauer/PycharmProjects/NLP_Bioread_Reddit_2025W/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/paulsilberbauer/PycharmProjects/NLP_Bioread_Reddit_2025W/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-23 00:35:43 INFO: Downloaded file to /Users/paulsilberbauer/stanza_resources/resources.json
2025-10-23 00:35:43 INFO: Downloading default packages for language: de (German) ...
2025-10-23 00:35:45 INFO: File exists: /Users/paulsilberbauer/stanza_resources/de/default.zip
2025-10-23 00:35:49 INFO: Finished downloading models and saved to /Users/paulsilberbauer/stanza_resource

,sentence_id,token_id,text,lemma,upos,xpos,feats,head,deprel
0,1,1,Angela,Angela,PROPN,NE,Case=Nom|Gender=Fem|Number=Sing,4,nsubj
1,1,2,Merkel,Merkel,PROPN,NE,Case=Nom|Gender=Fem|Number=Sing,1,flat
2,1,3,war,sein,AUX,VAFIN,Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbF...,4,cop
3,1,4,Präsidentin,Präsidentin,NOUN,NN,Case=Nom|Gender=Fem|Number=Sing,0,root
4,1,5,.,.,PUNCT,$.,None,4,punct
